In [15]:
import pandas as pd
import nltk 
import numpy as np
import re
from nltk.stem import wordnet # to perform lemmitization
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer,PorterStemmer
#from sklearn.feature_extraction.text import CountVectorizer # to perform bow
from sklearn.feature_extraction.text import TfidfVectorizer # to perform tfidf
from nltk import pos_tag # for parts of speech
from sklearn.metrics import pairwise_distances # to perfrom cosine similarity
from nltk import word_tokenize # to create tokens
from nltk.corpus import stopwords # for stop words

In [16]:
df=pd.read_excel('dialog_chatbot.xlsx')
df.head(20)

Context  \
0   Tell me about your personality   
1        I want to know you better   
2                  Define yourself   
3                Describe yourself   
4           tell me about yourself   
5                    all about you   
6     tell me some stuff about you   
7        talk some stuff about you   
8              talk about yourself   
9                   about yourself   
10                     who are you   
11              introduce yourself   
12   I want to know more about you   
13                    what are you   
14        what is your personality   
15                   say about you   
16               tell me about you   
17                why are you here   
18                             NaN   
19            are you 21 years old   

                                        Text Response  
0         Just think of me as the ace up your sleeve.  
1       I can help you work smarter instead of harder  
2   Hello, I am the chatbot of Pentol Resto Medan....  
3                                                 NaN  
4                                                 NaN  
5                                                 NaN  
6                                                 NaN  
7                                                 NaN  
8                                                 NaN  
9                                                 NaN  
10                                                NaN  
11                                                NaN  
12                                                NaN  
13                                                NaN  
14                                                NaN  
15                                                NaN  
16                                                NaN  
17                                                NaN  
18                                                NaN  
19  I'm a relatively new bot, but I'm wise beyond ...

In [17]:
df.shape[0] # returns the number of rows in dataset

1649

In [18]:
df.ffill(axis = 0,inplace=True) # fills the null value with the previous value.
df

Context  \
0                      Tell me about your personality   
1                           I want to know you better   
2                                     Define yourself   
3                                   Describe yourself   
4                              tell me about yourself   
5                                       all about you   
6                        tell me some stuff about you   
7                           talk some stuff about you   
8                                 talk about yourself   
9                                      about yourself   
10                                        who are you   
11                                 introduce yourself   
12                      I want to know more about you   
13                                       what are you   
14                           what is your personality   
15                                      say about you   
16                                  tell me about you   
17                                   why are you here   
18                                   why are you here   
19                               are you 21 years old   
20                                   what is your age   
21                                    how old are you   
22                                       age of yours   
23                           how old is your platform   
24                                   tell me your age   
25                          I'd like to know your age   
26                          I'd like to know your age   
27                                   you are annoying   
28                                I find you annoying   
29                         you're incredibly annoying   
...                                               ...   
1619       does pentol resto medan have social media?   
1620   does pentol resto medan have facebook account?   
1621  does pentol resto medan have Instagram account?   
1622    does pentol resto medan have Twitter account?   
1623    does pentol resto medan have Twitter account?   
1624                       What is the favourite one?   
1625                           Tell me the best food!   
1626                           what is the best food?   
1627                           what is the best food?   
1628          What is the ingredients of Pentol Ayam?   
1629         What is the ingredients of Pentol Puyuh?   
1630          What is the ingredients of Pentol Sapi?   
1631                What is the ingredients of Bakso?   
1632               What is the ingredients of Siomay?   
1633          What is the ingredients of Nasi Goreng?   
1634           What is the ingredients of Mie Goreng?   
1635                 What is the ingredients of Soto?   
1636             What is the ingredients of Sup Ayam?   
1637             What is the ingredients of Mie Ayam?   
1638             What is the ingredients of Mie Ayam?   
1639   what are the nutritional facts of Pentol Ayam?   
1640  what are the nutritional facts of Pentol Puyuh?   
1641   what are the nutritional facts of Pentol Sapi?   
1642         what are the nutritional facts of Bakso?   
1643        what are the nutritional facts of Siomay?   
1644   what are the nutritional facts of Nasi Goreng?   
1645    what are the nutritional facts of Mie Goreng?   
1646          what are the nutritional facts of Soto?   
1647      what are the nutritional facts of Sup Ayam?   
1648      what are the nutritional facts of Mie Ayam?   

                                          Text Response  
0           Just think of me as the ace up your sleeve.  
1         I can help you work smarter instead of harder  
2     Hello, I am the chatbot of Pentol Resto Medan....  
3     Hello, I am the chatbot of Pentol Resto Medan....  
4     Hello, I am the chatbot of Pentol Resto Medan....  
5     Hello, I am the chatbot of Pentol Resto Medan....  
6     Hello, I am the chatbot of Pentol Resto Medan....  
7     Hello, I am the chatbot of Pentol Resto M

In [19]:
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer() 

In [20]:
def preprocess(sentence):
    sentence=str(sentence)
    sentence = sentence.lower()
    sentence=sentence.replace('{html}',"") 
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', sentence)
    rem_url=re.sub(r'http\S+', '',cleantext)
    rem_num = re.sub('[0-9]+', '', rem_url)
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(rem_num)  
    filtered_words = [w for w in tokens if len(w) > 2 if not w in stopwords.words('english')]
    stem_words=[stemmer.stem(w) for w in filtered_words]
    tags_list=pos_tag(tokens,tagset=None)
    lemma_words=[lemmatizer.lemmatize(w) for w in stem_words]
    for token,pos_token in tags_list:
        if pos_token.startswith('V'):  # Verb
            pos_val='v'
        elif pos_token.startswith('J'): # Adjective
            pos_val='a'
        elif pos_token.startswith('R'): # Adverb
            pos_val='r'
        else:
            pos_val='n' # Noun
        lema_token=lemmatizer.lemmatize(token,pos_val) # performing lemmatization
        lemma_words.append(lema_token) # appending the lemmatized token into a list
    return " ".join(filtered_words)


In [21]:
df['lemmatized_text']=df['Context'].apply(preprocess) # applying the fuction to the dataset to get clean text
df.tail(15)

Context  \
1634           What is the ingredients of Mie Goreng?   
1635                 What is the ingredients of Soto?   
1636             What is the ingredients of Sup Ayam?   
1637             What is the ingredients of Mie Ayam?   
1638             What is the ingredients of Mie Ayam?   
1639   what are the nutritional facts of Pentol Ayam?   
1640  what are the nutritional facts of Pentol Puyuh?   
1641   what are the nutritional facts of Pentol Sapi?   
1642         what are the nutritional facts of Bakso?   
1643        what are the nutritional facts of Siomay?   
1644   what are the nutritional facts of Nasi Goreng?   
1645    what are the nutritional facts of Mie Goreng?   
1646          what are the nutritional facts of Soto?   
1647      what are the nutritional facts of Sup Ayam?   
1648      what are the nutritional facts of Mie Ayam?   

                                          Text Response  \
1634  The basic ingredients of the Mie Goreng consis...   
1635  The basic ingredients of the Soto consist of:\...   
1636  The basic ingredients of the Sup Ayam consist ...   
1637  The basic ingredients of the Mie Ayam consist ...   
1638  The basic ingredients of the Mie Ayam consist ...   
1639  The nutritional facts of Pentol Ayam are as fo...   
1640  The nutritional facts of Pentol Puyuh are as f...   
1641  The nutritional facts of Pentol Sapi are as fo...   
1642  The nutritional facts of Bakso are as follows:...   
1643  The nutritional facts of Siomay are as follows...   
1644  The nutritional facts of Nasi Goreng are as fo...   
1645  The nutritional facts of Mie Goreng are as fol...   
1646  The nutritional facts of Soto are as follows:\...   
1647  The nutritional facts of Sup Ayam are as follo...   
1648  The nutritional facts of Mie Ayam are as follo...   

                     lemmatized_text  
1634          ingredients mie goreng  
1635                ingredients soto  
1636            ingredients sup ayam  
1637            ingredients mie ayam  
1638            ingredients mie ayam  
1639   nutritional facts pentol ayam  
1640  nutritional facts pentol puyuh  
1641   nutritional facts pentol sapi  
1642         nutritional facts bakso  
1643        nutritional facts siomay  
1644   nutritional facts nasi goreng  
1645    nutritional facts mie goreng  
1646          nutritional facts soto  
1647      nutritional facts sup ayam  
1648      nutritional facts mie ayam

# tf-idf

In [22]:
Question1 ='Tell me about yourself.'

In [24]:
Question_lemma1 = preprocess(Question1)
Question_tfidf = tfidf.transform([Question_lemma1]).toarray() # applying tf-idf

NameError: name 'tfidf' is not defined